# Schach Spiel mit importieren S_n

## Imports, die für dieses Notebook benötigt werden

In [51]:
import chess
from IPython.display import clear_output
from IPython.display import display
import json
import random
from stockfish import Stockfish
import time


## Ausgangssituation

Für dieses Board wurde die S_n Sequenz berechnet.

In [54]:
fen = "8/8/8/8/4k3/8/8/K6R w - - 0 1"
#fen = "4k3/5R2/4K3/8/8/8/8/8 w - - 0 1"
original_board = chess.Board(fen)

## Import der Daten

Die Daten wurden als FEN in der JSON-Datei Serialisiert.
Zum Initialisieren der Liste werden alle FENs gelesen und Board-Objekte erstellt.

In [55]:
S_n_sequence_new = []
f = open("S_n_seq_12_01.json", "r")
tmp = json.loads(f.read())
for item in tmp:
    tmp_list = []
    for board in item:
        tmp_list.append(chess.Board(board))
    S_n_sequence_new.append(tmp_list)
f.close()

## findBoardInSequence Hilfsfunktion

Diese Funktion durchsucht eine S_n Sequenz nach dem ersten Vorkommen eines übergebenen Board-Objekts.

Funktions-Argumente:
* situation: Das Board (als Objekt), welches gefunden werden soll
* sequence: Die S_n-Sequenz, in welcher das Board gesucht wird

Ergebnis der Ausführung:
* Die Funktion hat zwei mögliche Rückgaben:
  * Ein Tupel mit S_n Index (z.B. S_3) und Board-Index (z.B. 100).
    Dieses Tupel drückt aus, wo in der Sequenz das Board gefunden wurde.
  * Das Tupel (-1,-1). Dies drückt aus, dass das Board nicht gefunden wurde.

Nebeneffekte:
Die Funktion verändert keinen der übergebenen Parameter.

Algorithmus:
1. Über die S_n-Sequenz iterieren.
2. Über jedes Board in einem spezifischem S_n iterieren.
3. Das Board mit dem _situation_ Objekt vergleichen.
    1. Wenn das Board übereinstimmt, die Indizes zurückgeben.
    2. Wenn das Board nicht übereinstimmt, weitersuchen.

In [56]:
def findBoardInSequence(situation, sequence):
    board_str = (situation.turn, situation.__str__())

    for i in range(len(sequence)):
        for j in range(len(sequence[i])):
            item = sequence[i][j]
            if board_str == (item.turn, item.__str__()):
                return i, j
    return -1,-1

## Finden des ersten Boards in der S_n-Sequenz

Mithilfe der zuvor definierten _findBoardInSequence_ Funktion wird das *original_board* in der *S_n_sequence* gesucht.

In [57]:
s_index = 0
board_index = 0

s_index, board_index = findBoardInSequence(original_board, S_n_sequence_new)

#Print search result
print("S" + str(s_index) + " - Board: " + str(board_index))

S31 - Board: 436


## Berechnen der Züge bis zum Spielende
Diese Berechnung erfolgt mit zufälligen Zügen für den Spieler "Schwarz".

Voraussetzungen:
  * Die vorherige Zelle muss ausgeführt sein; Es müssen ein s_index und ein board_index gesetzt sein.

Ablauf:
  * Das aktuelle Board kopieren (Auswählen aus S_n und kopieren)
  * Überprüfen ob Weiß oder Schwarz am Zug ist:
    * Wenn Weiß:
      * Alle legal moves ausprobieren und überprüfen in welcher S_n Menge das Board sich hinterher befindet.
      * Den legal Move, mit dem geringsten n auswählen
    * Wenn Schwarz:
      * Einen zufälligen (legal-)Move durchführen
      * Überprüfen in welcher S_n Menge das Board sich nun befindet
  * s_index und board_index aktualisieren
  * Wiederholen bis, das Board in S_0 ist (s_index = 0) oder kein Move gefunden wurde (s_index = -1)

In [58]:
stockfish = Stockfish("./stockfish_14.1/stockfish_14.1_win_x64_avx2.exe")

moves = []
while s_index > 0:
    curr_board = S_n_sequence_new[s_index][board_index].copy()
    stockfish.set_fen_position(curr_board.fen())
    if curr_board.turn:
        print("---White:---")
        print("Starting in S" + str(s_index))
        placement_dict = {}
        for move in curr_board.legal_moves:
            curr_board.push(move)
            _tmp, board_index_tmp = findBoardInSequence(curr_board, [S_n_sequence_new[s_index - 1]])
            s_index_tmp = s_index - 1
            if board_index_tmp != -1:
                print("    S" + str(s_index_tmp) + " - Board: " + str(board_index_tmp))
                placement_dict[(move,s_index_tmp,board_index_tmp)] = s_index_tmp
            curr_board.pop()
        print(placement_dict)
        move, s_index, board_index = min(placement_dict, key=placement_dict.get)
        print(move)
        moves.append(move)
        print("Ended in S" + str(s_index))

    else:
        print("---Black:---")
        print("Starting in S" + str(s_index))
        #move_list = list(curr_board.legal_moves)
        #move_index = random.randint(0, len(move_list) - 1)
        move = chess.Move.from_uci(stockfish.get_best_move())
        curr_board.push(move)
        s_index, board_index = findBoardInSequence(curr_board, S_n_sequence_new[:s_index])
        print("    S" + str(s_index) + " - Board: " + str(board_index))
        if s_index == -1:
            print("No solution")
            break
        print(move)
        moves.append(move)
        print("Ended in S" + str(s_index))


print(str(len(moves)) + " Züge bis zum Gewinn bestimmt")

---White:---
Starting in S31
    S30 - Board: 1694
    S30 - Board: 12126
{(Move.from_uci('h1d1'), 30, 1694): 30, (Move.from_uci('a1b2'), 30, 12126): 30}
h1d1
Ended in S30
---Black:---
Starting in S30
    S29 - Board: 718
e4f5
Ended in S29
---White:---
Starting in S29
    S28 - Board: 38
{(Move.from_uci('d1e1'), 28, 38): 28}
d1e1
Ended in S28
---Black:---
Starting in S28
    S27 - Board: 1148
f5g4
Ended in S27
---White:---
Starting in S27
    S26 - Board: 14182
    S26 - Board: 17667
{(Move.from_uci('e1f1'), 26, 14182): 26, (Move.from_uci('a1b2'), 26, 17667): 26}
e1f1
Ended in S26
---Black:---
Starting in S26
    S23 - Board: 1701
g4h5
Ended in S23
---White:---
Starting in S23
    S22 - Board: 7736
{(Move.from_uci('f1g1'), 22, 7736): 22}
f1g1
Ended in S22
---Black:---
Starting in S22
    S21 - Board: 2023
h5h4
Ended in S21
---White:---
Starting in S21
    S20 - Board: 5331
    S20 - Board: 57
    S20 - Board: 5328
{(Move.from_uci('a1b2'), 20, 5331): 20, (Move.from_uci('a1a2'), 20, 57):

## Ergebnis mit Stockfish Ergebnis vergleichen

In [59]:
compare_board = chess.Board()
compare_board.set_fen(fen)
compare_moves = []

while not compare_board.is_game_over():
    stockfish.set_fen_position(compare_board.fen())
    next_move = chess.Move.from_uci(stockfish.get_best_move())
    compare_board.push(next_move)
    compare_moves.append(next_move)

print("Stockfish ended the Game in " + str(len(compare_moves)) + " turns.")

Stockfish ended the Game in 29 turns.


## Anzeigen des Ergebnisses
Ergebnisse anzeigen, in dem mit 2 Sekunden Verzögerung alle Moves durchgeführt werden.

In [ ]:
test_board = original_board.copy()
display(test_board)
time.sleep(2)
for move in moves:
    test_board.push(move)
    clear_output(wait=True)
    display(test_board)
    time.sleep(2)

Die Funktion `move_in_sequence` überprüft für ein gegebenes Brett, ob dieses auch in der gegebenen S_n Menge vorhanden ist.
Hierfür werden folgende Parameter verwendet:
- `board_to_check`: Das Brett, das in der Menge `sequence` vorhanden sein soll.
- `sequence`: Die Menge, in der `board_to_check` auffindbar sein soll.

Als Ergebnis wird ein boolescher Wert zurückgegeben.

In [60]:
def move_in_sequence(board_to_check, sequence):
    if findBoardInSequence(board_to_check, [sequence]) != (-1,-1):
        return True
    else:
        return False

Bei `sequenz_index` wird für jedes Board einer S_n Menge überprüft, ob Stockfish oder die zuvor definierten Mengen das
Board effizienter lösen. Dabei gilt es für den Vergleich zu beachten, dass folgende Vorgehensweise verwendet wird.
- `Stockfish`: Für sowohl schwarz, als auch für weiß, werden die Züge mit der Funktion `Stockfish().get_best_move()` bestimmt.
- `KI`: Das n, das überprüft wird, wird für den Vergleich genutzt, da dies den schlechtesten Fall darstellt.

Die Methode, die für den Vergleich mit Stockfish angewandt wurde, birgt die Gefahr, dass durch Stockfish für schwarz
nicht der beste Zug ausgewählt wird und dadurch das Ergebnis beeinflusst. Diese Gefahr erhöht sich vor allem dann, wenn
sich die Züge bis zu einem Schachmatt erhöhen, da bei jedem Zug von schwarz die Gefahr besteht, dass nicht der beste Zug
ausgewählt wird.

Die Funktion erhält als Parameter:
- `sequenz_index`: Die Nummer der Menge, für die alle Bretter überprüft werden sollen.
(Bsp.: S_10 -> `sequence_index = 10`)

In der Konsole wird nach den Berechnungen dem Nutzer angezeigt, zu wie viel % Stockfish und die selbstgeschriebene KI
bessere Ergebnisse erzielt hat. Außerdem wird noch angezeigt zu welchem Teil sowohl Stockfish, als auch die KI gleiche
Ergebnisse erzielt haben.

In [72]:
def compare_sequence_stockfish(sequenz_index):
    s_n = S_n_sequence_new[sequenz_index]
    move_count_list = []
    diff_list = []
    for board_it in s_n:
        seq_index = sequenz_index
        cmp_board = board_it.copy()
        _u, b_index = findBoardInSequence(board_it, [s_n])
        # print(seq_index, b_index)
        move_count = 0
        # print(move_in_sequence(board_it, s_n))
        if move_in_sequence(board_it, s_n):
            while seq_index > 0:
                # White
                board_to_use = S_n_sequence_new[seq_index][board_index].copy()

                if board_to_use.turn:
                    # print("White")
                    for p_move in board_to_use.legal_moves:
                        # print(p_move)
                        board_to_use.push(p_move)
                        # print("B function: ", b_index, seq_index)
                        s_tmp, b_index = findBoardInSequence(board_to_use, [S_n_sequence_new[seq_index-1]])
                        # print("A function: ", b_index, seq_index)
                        if b_index != -1:
                            seq_index -= 1
                            # print(b_index, seq_index)
                            move_count += 1
                            break
                        board_to_use.pop()
                # Black
                else:
                    # print("Black")
                    stockfish.set_fen_position(board_to_use.fen())
                    nxt_move = chess.Move.from_uci(stockfish.get_best_move())
                    # print(nxt_move)
                    board_to_use.push(nxt_move)
                    move_count += 1
                    seq_index, b_index = findBoardInSequence(board_to_use, S_n_sequence_new[:seq_index])
                    # print(b_index, seq_index)
        else:
            print("Couldn't find board!")

        cmp_move_count = 0
        while not cmp_board.is_game_over():
            stockfish.set_fen_position(cmp_board.fen())
            nxt_move = chess.Move.from_uci(stockfish.get_best_move())
            cmp_board.push(nxt_move)
            cmp_move_count += 1
        print("Moves KI: " + str(move_count), "Moves Stockfish: " + str(cmp_move_count))
        move_count_list.append((move_count, cmp_move_count, move_count - cmp_move_count))
        diff_list.append(move_count - cmp_move_count)
    ki_better = 0
    stockfish_better = 0
    equal = 0
    for value in diff_list:
        if value == 0:
            equal += 1
        elif value < 0:
            ki_better += 1
        else:
            stockfish_better += 1
    cmp_values = len(diff_list)
    print("Stockfish war zu " + str(round((stockfish_better/cmp_values) * 100, 2)) + "% besser")
    print("Die KI war zu " + str(round((ki_better/cmp_values) * 100, 2)) + "% besser")
    print("Stockfish und die KI haben zu " + str(round((equal/cmp_values) * 100, 2)) + "% die gleichen Ergebnisse erzielt")

Als Beispiel wird hier der Vergleich zwischen der KI und Stockfish für die Menge S_20 vorgenommen.

In [76]:
compare_sequence_stockfish(6)

Moves KI: 6 Moves Stockfish: 6
Moves KI: 6 Moves Stockfish: 20
Moves KI: 6 Moves Stockfish: 20
Moves KI: 6 Moves Stockfish: 14
Moves KI: 6 Moves Stockfish: 16
Moves KI: 6 Moves Stockfish: 20
Moves KI: 6 Moves Stockfish: 20
Moves KI: 6 Moves Stockfish: 20
Moves KI: 6 Moves Stockfish: 6
Moves KI: 6 Moves Stockfish: 16
Moves KI: 6 Moves Stockfish: 14
Moves KI: 6 Moves Stockfish: 20
Moves KI: 6 Moves Stockfish: 6
Moves KI: 6 Moves Stockfish: 8
Moves KI: 6 Moves Stockfish: 6
Moves KI: 6 Moves Stockfish: 8
Moves KI: 6 Moves Stockfish: 8
Moves KI: 6 Moves Stockfish: 8
Moves KI: 6 Moves Stockfish: 6
Moves KI: 6 Moves Stockfish: 6
Moves KI: 6 Moves Stockfish: 8
Moves KI: 6 Moves Stockfish: 6
Moves KI: 6 Moves Stockfish: 6
Moves KI: 6 Moves Stockfish: 6
Moves KI: 6 Moves Stockfish: 6
Moves KI: 6 Moves Stockfish: 14
Moves KI: 6 Moves Stockfish: 14
Moves KI: 6 Moves Stockfish: 20
Moves KI: 6 Moves Stockfish: 14
Moves KI: 6 Moves Stockfish: 14
Moves KI: 6 Moves Stockfish: 6
Moves KI: 6 Moves Stockf

KeyboardInterrupt: 

Ein Testszenario sieht auch vor alle zuvor bestimmten Boards mit dem Lösungsweg von Stockfish zu vergleichen. Hierfür
geht die Funktion `compare_all_sequences` alle S_n-Mengen durch und vergleicht diese jeweils mit der Lösung von
Stockfish.

In [ ]:
def compare_all_sequences():
    for i in range(len(S_n_sequence_new)):
        print("Comparing S_" + str(i) + "...")
        compare_sequence_stockfish(i)

In [ ]:
compare_all_sequences()